In [5]:
import pickle
import pandas as pd
import os

In [6]:
with open('sp500tickers.pickle','rb') as f:
    tickers = pickle.load(f)

In [10]:
if not os.path.exists('stock_return'):
    os.makedirs('stock_return')
for ticker in tickers:
    df = pd.read_csv('stock_dfs/{}.csv'.format(ticker))
    df['Date'] = df['Date'].str.split('-')
    df['year'] = df['Date'].str[0]
    df['month'] = df['Date'].str[1]
    df_month = pd.DataFrame()
    n = len(df)
    for i in range(n-1):
        if (int(df.loc[i,'month'])!=int(df.loc[i+1,'month'])):
            df_month = df_month.append(df.loc[i])
    df_month = df_month.reset_index(inplace=False)    
    n = len(df_month)
    df_quart = pd.DataFrame()
    for i in range(n):
        if (int(df_month.loc[i,'month'])%3==0):
            df_quart = df_quart.append(df_month.loc[i])
            df_quart.loc[i,'quart'] = int(df_month.loc[i,'month'])//3
    df_quart = df_quart.reset_index(inplace = False)
    n = len(df_quart)
    df_quart.loc[0,'return'] = 1
    for i in range(1,n):
        stock_return = df_quart.loc[i,'Adj Close']/df_quart.loc[i-1,'Adj Close']
        df_quart.loc[i,'return'] = df_quart.loc[i,'Adj Close']/df_quart.loc[i-1,'Adj Close']
        df_quart.loc[i,'last_return'] = df_quart.loc[i-1,'return']
    df_quart = df_quart.drop(['level_0','Adj Close','Close','Date','High','Low','Open','Volume','index','month'],axis=1)
    df_quart = df_quart.drop(0,axis=0)
    df_quart.to_csv('stock_return/{}.csv'.format(ticker))